<a href="https://colab.research.google.com/github/jl749/CJ_VRP/blob/master/JM/final_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
import os

# drive.mount('/gdrive/', force_remount=True)
# PATH = "/gdrive/MyDrive/CJ/2. 택배 배송 데이터/"  # 택배 배송 데이터 PATH
PATH = "C:/Users/user/Desktop/CJ/2. 택배 배송 데이터/"  # 택배 배송 데이터 PATH

takbe_CSVs = os.listdir(PATH)

takbe_CSVs

In [ ]:
import pandas as pd

df1 = pd.read_excel(PATH + takbe_CSVs[0])
df2 = pd.read_excel(PATH + takbe_CSVs[1])
df3 = pd.read_excel(PATH + takbe_CSVs[2])
df4 = pd.read_excel(PATH + takbe_CSVs[3])
df5 = pd.read_excel(PATH + takbe_CSVs[4])
df6 = pd.read_excel(PATH + takbe_CSVs[5])
df7 = pd.read_excel(PATH + takbe_CSVs[6])
df8 = pd.read_excel(PATH + takbe_CSVs[7])
df9 = pd.read_excel(PATH + takbe_CSVs[8])
df10 = pd.read_excel(PATH + takbe_CSVs[9])
df11 = pd.read_excel(PATH + takbe_CSVs[10])
df12 = pd.read_excel(PATH + takbe_CSVs[11])
df13 = pd.read_excel(PATH + takbe_CSVs[12])
df14 = pd.read_excel(PATH + takbe_CSVs[13])

df_all = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14])

In [ ]:
df_all.head(3)

In [ ]:
df = df_all.drop(['Unnamed: 0', '구분명', '집화일자', '집배일자', '운임명', '집화여부', '집배일자', '장비구분', '품목'], axis=1)

In [ ]:
df.head(3)

# SM groupby 로 택배 용량 확인

In [ ]:
df_SM = df[['SM명', '배달일자', '수량(BOX)', '운임']].groupby(['SM명', '배달일자']).sum()
df_SM['val'] = df_SM['운임'] / df_SM['수량(BOX)']
df_SM

In [9]:
# df_SM[(0, '2021-03-02')]

In [ ]:
df_SM.index

In [ ]:
import matplotlib.pyplot as plt

df_SM.plot(figsize=(30,10), kind='bar', y='val')
plt.axis('off')
plt.show()

In [ ]:
df.head(3)

In [ ]:
import re

def if_integer(string):
    if bool(re.search(r'(\d+)\s*-\s*(\d+)', string)) or bool(re.search(r'(\d+)', string)):
        return True
    return False

In [ ]:
df = df.dropna(axis = 0)

In [ ]:
df = df[df.받는분주소.apply(if_integer)]
df

#사용가능한 주소 추출

In [ ]:
def remwithre(text, there=re.compile(re.escape('(')+'.*')):
  return there.sub('', text)

In [ ]:
df['받는분주소'] = df['받는분주소'].apply(remwithre)
df

In [ ]:
df.reset_index(inplace=True)
del df['index']

In [ ]:
df

In [ ]:
from urllib.parse import quote_plus, urlencode
from urllib.request import urlopen, Request
import time
import json

def jusoAPI(keystr):
  url = 'http://www.juso.go.kr/addrlink/addrLinkApi.do'
  queryParams = '?' + urlencode(
      {quote_plus('currentPage'): '1', quote_plus('countPerPage'): '1', quote_plus('resultType'): 'json',
      quote_plus('keyword'): keystr, quote_plus('confmKey'): 'devU01TX0FVVEgyMDIxMTAwMjIzNDkxMzExMTcxNDg='})
  request = Request(url + queryParams)
  request.get_method = lambda: 'GET'  # default GET anyway
  while True:
    try:
        response_body = urlopen(request).read()
    except:
        print(f"CONNECTION ERR ..... retry after 60sec")
        time.sleep(60)
        continue
    break

  root_json = json.loads(response_body)
  total = root_json['results']['common']['totalCount']

  if total == '1':
    ad = root_json['results']['juso'][0]
    return ad['jibunAddr'], ad['roadAddr']  # ['detBdNmList']
  else:
    return None

In [ ]:
jibun = pd.Series( data = [None] * df.shape[0] )
road = pd.Series( data = [None] * df.shape[0] )

index = 0
for ad in df['받는분주소']:
    tmp = jusoAPI(ad)
    if tmp is not None:
      jibun[index] = tmp[0]
      road[index] = tmp[1]
    else:
      arr = ad.split()
      count = 1
      while len(arr[ : -count]) > 2:
        tmp = jusoAPI(" ".join(arr[ : -count]))
        if tmp is not None:
          jibun[index] = tmp[0]
          road[index] = tmp[1]
          break
        time.sleep(0.1)
        count += 1
    index += 1
    print(f"index {index} succeed")

print(jibun)
print(road)

In [50]:
jibun[:20], road[:20]

(0     경기도 화성시 장안면 석포리 718-106
 1      경기도 화성시 장안면 석포리 785-14
 2       경기도 화성시 장안면 석포리 334-1
 3      경기도 화성시 장안면 석포리 448-24
 4       경기도 화성시 장안면 석포리 386-3
 5      경기도 화성시 장안면 석포리 614-24
 6         경기도 화성시 장안면 석포리 741
 7      경기도 화성시 장안면 석포리 718-54
 8     경기도 화성시 장안면 석포리 718-116
 9     경기도 화성시 장안면 석포리 718-134
 10      경기도 화성시 장안면 석포리 564-3
 11     경기도 화성시 장안면 석포리 675-15
 12      경기도 화성시 장안면 석포리 675-3
 13      경기도 화성시 장안면 석포리 406-1
 14     경기도 화성시 장안면 석포리 718-12
 15      경기도 화성시 장안면 석포리 614-4
 16      경기도 화성시 장안면 석포리 771-1
 17     경기도 화성시 장안면 석포리 763-17
 18       경기도 화성시 장안면 석포리 59-3
 19    경기도 화성시 장안면 석포리 785-242
 dtype: object, 0         경기도 화성시 장안면 매바위로366번길 8
 1            경기도 화성시 장안면 매바위로 347
 2          경기도 화성시 장안면 버들로 1232-4
 3           경기도 화성시 장안면 무봉길 200-6
 4         경기도 화성시 장안면 소개무골길 66-10
 5          경기도 화성시 장안면 돌서지길 96-27
 6           경기도 화성시 장안면 돌서지길 28-7
 7            경기도 화성시 장안면 매바위로 376
 8     경기도 화성시 장안면 매바위로366번길 68-13
 9        경기도 화성시 장안면 매바위로366번길 25
 10         경기도

In [31]:
df['지번'] = jibun
df['도로명주소'] = road
df

,수량(BOX),운임,집배시간,배달일자,SM명,받는분주소,지번,도로명주소
0,1,5000,190148,2021-03-03,57,경기도 화성시 장안면 매바위로366번길 8,경기도 화성시 장안면 석포리 718-106,경기도 화성시 장안면 매바위로366번길 8
1,1,3500,180605,2021-03-04,57,경기도 화성시 장안면 매바위로 347,경기도 화성시 장안면 석포리 785-14,경기도 화성시 장안면 매바위로 347
2,1,3500,175052,2021-03-05,57,경기도 화성시 장안면 버들로 1232-4,경기도 화성시 장안면 석포리 334-1,경기도 화성시 장안면 버들로 1232-4
3,1,5000,121446,2021-03-08,57,경기 화성시 장안면 무봉길 200-6,경기도 화성시 장안면 석포리 448-24,경기도 화성시 장안면 무봉길 200-6
4,1,5000,180605,2021-03-04,57,경기도 화성시 장안면 소개무골길 66-10,경기도 화성시 장안면 석포리 386-3,경기도 화성시 장안면 소개무골길 66-10
...,...,...,...,...,...,...,...,...
4037719,1,2500,113256,2021-04-29,170,경기 화성시 봉담읍 상봉길 35-7,None,None
4037720,1,2500,155117,2021-04-28,170,[183-16] 경기도 화성시 봉담읍 상봉길 35-10,None,None
4037721,1,6000,152249,2021-04-29,170,경기 화성시 봉담읍 샘마을1길 35-3,None,None
4037722,1,2500,111242,2021-04-27,170,경기 화성시 봉담읍 상봉길 35-17 써니빌4,None,None


In [32]:
df.dropna()

,수량(BOX),운임,집배시간,배달일자,SM명,받는분주소,지번,도로명주소
0,1,5000,190148,2021-03-03,57,경기도 화성시 장안면 매바위로366번길 8,경기도 화성시 장안면 석포리 718-106,경기도 화성시 장안면 매바위로366번길 8
1,1,3500,180605,2021-03-04,57,경기도 화성시 장안면 매바위로 347,경기도 화성시 장안면 석포리 785-14,경기도 화성시 장안면 매바위로 347
2,1,3500,175052,2021-03-05,57,경기도 화성시 장안면 버들로 1232-4,경기도 화성시 장안면 석포리 334-1,경기도 화성시 장안면 버들로 1232-4
3,1,5000,121446,2021-03-08,57,경기 화성시 장안면 무봉길 200-6,경기도 화성시 장안면 석포리 448-24,경기도 화성시 장안면 무봉길 200-6
4,1,5000,180605,2021-03-04,57,경기도 화성시 장안면 소개무골길 66-10,경기도 화성시 장안면 석포리 386-3,경기도 화성시 장안면 소개무골길 66-10
...,...,...,...,...,...,...,...,...
8209,1,5000,185424,2021-03-04,200,경기도 화성시 기안동 457-54,경기도 화성시 기안동 457-54,경기도 화성시 융건로 47-14(기안동)
8210,1,5000,191125,2021-03-03,200,경기도 화성시 배양남길40번길 19,경기도 화성시 기안동 20-185,경기도 화성시 배양남길40번길 19(기안동)
8211,1,3500,194004,2021-03-04,200,경기도 화성시 융건로140번길 3-3,경기도 화성시 배양동 69-48,경기도 화성시 융건로140번길 3-3(배양동)
8212,1,6000,190736,2021-03-06,200,경기도 화성시 융건로 90,경기도 화성시 배양동 76 배양초등학교,경기도 화성시 융건로 90(배양동)


In [ ]:
df.to_excel("address_PREPROCESSED.xlsx")